# Usage of `src.models.weather` module.

This notebook outlies the basic usage of the `src.models.weather` module. Used to define weather models and estimate values at different altitudes
 
**Requirements**
 - A csv with weather station data

**Helpful Links**
 - [Weather Models Wiki](https://github.com/Flight-Path-Analysis/FlightPathAnalysis/wiki/Weather_Models)


## Basic Setup

In [20]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
import numpy as np

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.models import weather
# Utilities package
from src.common import utils

root_path = os.path.normpath(root_path) # Path from this notebook to the root directory
config_path_from_root = os.path.normpath('config/config_tutorial.yml') # Path from root to the desired config file
config_path = os.path.join(root_path, config_path_from_root) # Defining path from this notebook to config file

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=clear)

# Loading weather station data
data_directory = 'tutorial_data'
station_data = pd.read_csv(os.path.join(data_directory, 'station_data.csv'), index_col = 0)

## Compute Quantities From Weather Models

This code computes the desired quantities at different elevations from the weather models in the package.

The currently available weather models are:
 - Temperature
   - The temperature model is a simple Lapse Rate model, that is, the temperature decreases linearly with height.
 - Air Pressure
   - The air pressure model is estimated from the Temperature model, where:
$$P(h) = A exp\left(-\frac{g M}{R}\int_0^h\frac{1}{T(h')}dh'\right)$$
   - All the constants can be found on the `models` seciton in the config file
 - Air Density
   - The air density model is estimated from the temperature and pressure models simply by:
   $$\rho(h) = \frac{P(h)}{R_d\times T(h)}$$
 - Clouds
   - The cloud coverage model is an interpolation of the sky condition as reported by the `skycX` and `skylX` variables. These are mapped to a number between 0 and 1 and interpolated.
   - Extrapolations are clipped to the nearest valid entry.
   - If no valid entry, return 0

In [19]:
height = 100
row = station_data.iloc[0]

LR = config['models']['temperature']['default-coefficient']
g = config['models']['constants']['gravitational-acceleration']
R = config['models']['constants']['gas-constant']
M = config['models']['constants']['molar-mass-air']
P0 = config['models']['constants']['atm-pressure']
Rd = config['models']['constants']['specific-gas-constant']

temperature_at_height = weather.temperature_model(height, row['elevation'], row['tmpf'], LR)
air_pressure_at_height = weather.air_pressure_model(height, row['elevation'], row['tmpf'], LR, g, R, M, P0)
air_density_at_height = weather.air_density_model(height, row['elevation'], row['tmpf'], LR, g, R, M, P0, Rd)
clouds_at_height = weather.cloud_model(height,
                                       row[['skyc1', 'skyc2', 'skyc3', 'skyc4']].apply(pd.to_numeric, errors='coerce').values,
                                       row[['skyl1', 'skyl2', 'skyl3', 'skyl4']].apply(pd.to_numeric, errors='coerce').values,
                                       config)
print(f'Temperature in F at {height} meters: {round(temperature_at_height,2)}')
print(f'Air Pressure in millibars at {height} meters: {round(air_pressure_at_height,2)}')
print(f'Air Density in Kg/m^3 at {height} meters: {round(air_density_at_height,2)}')
print(f'Cloud Level at {height} meters: {round(clouds_at_height,2)}')


Temperature in F at 100 meters: 34.48
Air Pressure in millibars at 100 meters: 1000.7
Air Density in Kg/m^3 at 100 meters: 1.27
Cloud Level at 100 meters: 0.0
